In [1]:
import sys, os, importlib, math, multiprocessing, boto3, pickle
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.ntlMisc as ntl
import GOSTrocks.mapMisc as mapMisc
from GOSTrocks.misc import tPrint

sys.path.append("../src")
import h3_helper

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [9]:
# Define S3 parameters
bucket = 'wbg-geography01' 
prefix = 'Space2Stats/sample_data'
region = 'us-east-1'
s3client = boto3.client('s3', region_name=region)
out_folder = "/home/wb411133/projects/Space2Stats/"

admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM0.shp"
inA = gpd.read_file(admin_bounds)
inA['ID'] = inA.index
inA.head()

,ISO_A3,ISO_A2,WB_A3,HASC_0,GAUL_0,WB_REGION,WB_STATUS,SOVEREIGN,NAM_0,geometry,ID
0,AFG,AF,AFG,AF,1,SAR,Member State,AFG,Afghanistan,"POLYGON ((70.04663 37.54360, 70.04676 37.54356...",0
1,ALB,AL,ALB,AL,3,ECA,Member State,ALB,Albania,"MULTIPOLYGON (((20.46186 41.55588, 20.46177 41...",1
2,AND,AD,ADO,AD,7,Other,Member State,AND,Andorra,"POLYGON ((1.46171 42.50602, 1.46176 42.50605, ...",2
3,ARE,AE,ARE,AE,255,MENA,Member State,ARE,United Arab Emirates,"MULTIPOLYGON (((53.83064 24.06414, 53.82975 24...",3
4,ARG,AR,ARG,AR,12,LCR,Member State,ARG,Argentina,"MULTIPOLYGON (((-58.43833 -34.00238, -58.44420...",4


In [3]:
# Extract/Generate H3 cells and download raster data for mapping generation
sel_iso3 = "KEN"
sel_admin = inA.loc[inA['ISO_A3'] == sel_iso3]


,ISO_A3,ISO_A2,WB_A3,HASC_0,GAUL_0,WB_REGION,WB_STATUS,SOVEREIGN,NAM_0,geometry,ID
260,KEN,KE,KEN,KE,133,AFR,Member State,KEN,Kenya,"MULTIPOLYGON (((33.94935 -0.33810, 33.95258 -0...",260


In [4]:
dir(h3_helper)

['CRS',
 'Polygon',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'ctx',
 'folium',
 'generate_h3_gdf',
 'generate_lvl0_lists',
 'generate_lvl1_lists',
 'gpd',
 'h3',
 'importlib',
 'json',
 'make_axes_locatable',
 'map_choropleth',
 'mapping',
 'matplotlib',
 'mpatches',
 'os',
 'pd',
 'pickle',
 'plt',
 'rasterio',
 'shapely',
 'static_map_h3',
 'sys',
 'tPrint',
 'tqdm',
 'unary_union',
 'urlopen']

In [15]:
out_path = f's3://{bucket}/{prefix}/{sel_iso3}'

for h3_lvl in [5,6,7]:
    out_file = os.path.join(out_path, f"h3_lvl{h3_lvl}_cells.geoparquet")
    try:
        gpd.read_file(out_file)
    except:
        h3_data = h3_helper.generate_h3_gdf(sel_admin, h3_lvl)
        h3_data.to_parquet(out_file)


Generating h3 grid level 5: 100%|██████████| 365/365 [00:01<00:00, 224.11it/s]
<ipython-input-15-7ed586aa6e45>:9: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  h3_data.to_parquet(out_file)
Generating h3 grid level 7: 100%|██████████| 365/365 [00:41<00:00,  8.88it/s] 


# Extract geospatial data

In [16]:
ntl_files = ntl.aws_search_ntl()
sel_file = ntl_files[0]

raster_path = "/home/wb411133/temp/s2s_temp"
if not os.path.exists(raster_path):
    os.makedirs(raster_path)

out_file = os.path.join(raster_path, os.path.basename(sel_file))
rMisc.clipRaster(rasterio.open(sel_file), sel_admin, out_file)

[array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32),
 {'driver': 'GTiff',
  'dtype': 'float32',
  'nodata': None,
  'width': 1921,
  'height': 2246,
  'count': 1,
  'crs': CRS.from_epsg(4326),
  'transform': Affine(0.0041666667, 0.0, 33.90625171125001,
         0.0, -0.0041666667, 4.631249437050002)}]